<a href="https://colab.research.google.com/github/DigiBP/Team-Chillon-Castle/blob/master/prescription_parser_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Input
Upload file with text.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Folie1.jpeg to Folie1.jpeg


### Install Pre-requisites
- tesseract linux binary
- pytesseract

In [2]:
!sudo apt install tesseract-ocr -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 14 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,569 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [3]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=17d8b9f124b7d11cd644bc7d12943df07076358278092eb4d1dec4b7d8dfed36
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract


### OCR

In [4]:
import pytesseract
from PIL import Image
file_content = pytesseract.image_to_string(Image.open('Folie1.jpeg'))

### Post-processing to Google Sheet

In [5]:
# Run this cell only once
from collections import OrderedDict
final_prescriptions = OrderedDict()

In [6]:
def parse_prescription(file_content):
  file_content_str = [elem.strip() for elem in file_content.split('\n')]
  prescription_row = {}
  for elem in map(str,file_content_str):
    if ':' in elem:
      key, val = elem.split(':')
      prescription_row[key] = val
  return prescription_row



In [7]:
# file_content = open('out.txt.txt', 'rb').readlines()
prescription_row=parse_prescription(file_content)

In [8]:
final_prescriptions[len(final_prescriptions)] = prescription_row

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame.from_dict(final_prescriptions, orient="index")
df.to_excel("prescriptions_dict.xlsx", 'Sheet1', index=False)
df

,Name,Zip,ePharmacy,Mail,Drug,Plan
0,Annika,4058,no,anni.rappi@gmail.com,Rocephin,1-0-1


In [11]:
#Export to Google Sheets / Part 1 Auth
!pip install --upgrade --quiet gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [18]:
#Export to Google Sheets / Part 2 Export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

patient_data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1tbLYRSAfDMbr8cueQInhWVA6ISAD2c8kc9X7EonvOX8/edit#gid=0')
ws1 = patient_data.get_worksheet(0)
ws1.update_cell(2,2,df.iloc[0]['Name'])
ws1.update_cell(2,3,df.iloc[0]['Zip'])
ws1.update_cell(2,4,df.iloc[0]['ePharmacy'])
ws1.update_cell(2,5,df.iloc[0]['Mail'])
ws1.update_cell(2,6,df.iloc[0]['Drug'])
ws1.update_cell(2,7,df.iloc[0]['Plan'])

{'spreadsheetId': '1tbLYRSAfDMbr8cueQInhWVA6ISAD2c8kc9X7EonvOX8',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Tabellenblatt1!G2',
 'updatedRows': 1}